In [26]:
import pandas as pd
import xgboost as xgb
import numpy as np

#
#import sklearn
#import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [58]:
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")
sample.head(2)

,id,target
0,0,0
1,2,0


In [12]:
test_limpio = pd.read_csv("test_limpio.csv",encoding = "ISO-8859-1")
test_limpio_sin_texto = test_limpio.drop(['text','keyword','location'], axis=1)
test_limpio_sin_texto.head()

,id,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,the,a,to,in,of,...,plans to,police post,provoke outrage,read by,rescuers are,letras_seguidas,insultos_por_tweet,expresiones_por_tweet,kill,died
0,0,0,0,1,40,0,1,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1,2,0,0,1,70,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
2,3,0,0,1,100,1,1,0,0,0,...,0,0,0,0,0,2,0,0,0,0
3,9,0,0,1,40,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,11,0,0,1,50,0,0,0,1,0,...,0,0,0,0,0,2,0,0,0,0


In [13]:
train_limpio = pd.read_csv("train_limpio.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,the,a,to,in,...,plans to,police post,provoke outrage,read by,rescuers are,letras_seguidas,insultos_por_tweet,expresiones_por_tweet,kill,died
0,"['our', 'deeds', 'are', 'the', 'reason', 'of',...",1,0,0,1,70,1,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1,"['forest', 'fire', 'near', 'la', 'ronge', 'sas...",1,0,0,1,40,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [14]:
df_x=train_limpio.drop(['text', 'target'], axis=1)
df_x.head()

,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,the,a,to,in,of,and,...,plans to,police post,provoke outrage,read by,rescuers are,letras_seguidas,insultos_por_tweet,expresiones_por_tweet,kill,died
0,0,0,1,70,1,0,0,0,1,0,...,0,0,0,0,0,2,0,0,0,0
1,0,0,1,40,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,1,130,0,0,1,2,0,0,...,0,0,0,0,0,2,0,0,0,0
3,0,0,1,60,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,1,90,0,1,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0


In [15]:
df_y=train_limpio.target
df_y

0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
7104    0
7105    0
7106    0
7107    0
7108    0
7109    0
7110    0
7111    1
7112    0
7113    1
7114    1
7115    0
7116    1
7117    1
7118    1
7119    1
7120    1
7121    0
7122    1
7123    1
7124    1
7125    1
7126    1
7127    1
7128    1
7129    1
7130    1
7131    1
7132    1
7133    1
Name: target, Length: 7134, dtype: int64

In [16]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=4)

In [17]:
#rf= RandomForestClassifier(n_estimators=50)
#xg_reg = xgb.XGBClassifier(objective ='reg:squarederror', 
#                colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)
xgc = xgb.XGBClassifier()
xgc.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

## Error de entrenamiento

In [20]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [21]:
train_sin_texto = train_limpio.drop('text',axis=1)

In [31]:
for train_index,test_index in kf.split(train_sin_texto):
    x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
    y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
    xgc = xgb.XGBClassifier()
    xgc.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = xgc.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = xgc.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))
    
    
    
    
    #x_train = train.drop['target']
    #x_test = test.drop['target']
    #y_train = train.target
    #y_test = test.target
    #print(train,test)

('Error de entrenamiento: ', 0.7400450933553037)
('Error de test: ', 0.6834457100891257)
('Error de entrenamiento: ', 0.7390952328653286)
('Error de test: ', 0.6215307989152553)
('Error de entrenamiento: ', 0.7308019629425015)
('Error de test: ', 0.6471494509396161)
('Error de entrenamiento: ', 0.7388500970427941)
('Error de test: ', 0.6699634162307817)
('Error de entrenamiento: ', 0.7295441487724972)
('Error de test: ', 0.6564762564762565)


In [24]:
# tunning

In [32]:
#Optimizo los 
best_score = 0
best_n_estimators = 0
current_score=[]
for i in np.arange(10,200,10):
    print('Estoy en la iteracion',i)
    current_score = []
    for train_index,test_index in kf.split(train_sin_texto):
        x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
        y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
        xgc = xgb.XGBClassifier(n_estimators=i)
        xgc.fit(x_train, y_train)
       
        #error de test:
        pred = xgc.predict(x_test)
        score = f1_score(y_test.values, pred, average='macro')
        current_score.append(score)
        
    print('El error total es:', sum(current_score)/len(current_score))
    if(sum(current_score)/len(current_score)>best_score):
        best_score = sum(current_score)/len(current_score)
        best_n_estimators = i
    print('')
    

print('Termine todo y la mejor iteracion fue con n_estimators:', best_min_samples_splite)    

('Estoy en la iteracion', 10)
('El error total es:', 0.5945839325576713)

('Estoy en la iteracion', 20)
('El error total es:', 0.5861897731458747)

('Estoy en la iteracion', 30)
('El error total es:', 0.6061766724257984)

('Estoy en la iteracion', 40)
('El error total es:', 0.623062982179448)

('Estoy en la iteracion', 50)
('El error total es:', 0.6310091293171545)

('Estoy en la iteracion', 60)
('El error total es:', 0.6353892992370406)

('Estoy en la iteracion', 70)
('El error total es:', 0.6426818372263194)

('Estoy en la iteracion', 80)
('El error total es:', 0.6484446080914446)

('Estoy en la iteracion', 90)
('El error total es:', 0.6517975872773802)

('Estoy en la iteracion', 100)
('El error total es:', 0.655713126530207)

('Estoy en la iteracion', 110)
('El error total es:', 0.6560969067478734)

('Estoy en la iteracion', 120)
('El error total es:', 0.6590265758855425)

('Estoy en la iteracion', 130)
('El error total es:', 0.6604986850536589)

('Estoy en la iteracion', 140)
('El 

NameError: name 'best_min_samples_splite' is not defined

In [33]:
for i in np.arange(0.1,0.8,0.05):
    print(i)

0.1
0.15000000000000002
0.20000000000000004
0.25000000000000006
0.30000000000000004
0.3500000000000001
0.40000000000000013
0.45000000000000007
0.5000000000000001
0.5500000000000002
0.6000000000000002
0.6500000000000001
0.7000000000000002
0.7500000000000002


In [34]:
#Optimizo los 
best_score = 0
best_learning_rate = 0
current_score=[]
for i in np.arange(0.1,0.8,0.05):
    print('Estoy en la iteracion',i)
    current_score = []
    for train_index,test_index in kf.split(train_sin_texto):
        x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
        y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
        xgc = xgb.XGBClassifier(n_estimators=170, learning_rate=i)
        xgc.fit(x_train, y_train)
       
        #error de test:
        pred = xgc.predict(x_test)
        score = f1_score(y_test.values, pred, average='macro')
        current_score.append(score)
        
    print('El error total es:', sum(current_score)/len(current_score))
    if(sum(current_score)/len(current_score)>best_score):
        best_score = sum(current_score)/len(current_score)
        best_n_estimators = i
    print('')
    

print('Termine todo y la mejor iteracion fue con learning_rate:', best_learning_rate) 

('Estoy en la iteracion', 0.1)
('El error total es:', 0.6696071409718021)

('Estoy en la iteracion', 0.15000000000000002)
('El error total es:', 0.6840317020076526)

('Estoy en la iteracion', 0.20000000000000004)
('El error total es:', 0.6948791612358646)

('Estoy en la iteracion', 0.25000000000000006)
('El error total es:', 0.6980475826294187)

('Estoy en la iteracion', 0.30000000000000004)
('El error total es:', 0.7024620105294133)

('Estoy en la iteracion', 0.3500000000000001)
('El error total es:', 0.7003358039716753)

('Estoy en la iteracion', 0.40000000000000013)
('El error total es:', 0.7033887830346475)

('Estoy en la iteracion', 0.45000000000000007)
('El error total es:', 0.7023512591389872)

('Estoy en la iteracion', 0.5000000000000001)
('El error total es:', 0.7001311519196041)

('Estoy en la iteracion', 0.5500000000000002)
('El error total es:', 0.7067419362383354)

('Estoy en la iteracion', 0.6000000000000002)
('El error total es:', 0.70740219669265)

('Estoy en la iteraci

In [41]:
#Optimizo los 
best_score = 0
best_max_depth = 0
current_score=[]
for i in np.arange(1,6,1):
    print('Estoy en la iteracion',i)
    current_score = []
    for train_index,test_index in kf.split(train_sin_texto):
        x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
        y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
        xgc = xgb.XGBClassifier(n_estimators=170,learning_rate=0.6 ,max_depth=i)
        xgc.fit(x_train, y_train)
       
        #error de test:
        pred = xgc.predict(x_test)
        score = f1_score(y_test.values, pred, average='macro')
        current_score.append(score)
        
    print('El error total es:', sum(current_score)/len(current_score))
    if(sum(current_score)/len(current_score)>best_score):
        best_score = sum(current_score)/len(current_score)
        best_max_depth = i
    print('')
    

print('Termine todo y la mejor iteracion fue con best_max_depth:', best_max_depth) 

('Estoy en la iteracion', 1)
('El error total es:', 0.6920187769437961)

('Estoy en la iteracion', 2)
('El error total es:', 0.7032492023962751)

('Estoy en la iteracion', 3)
('El error total es:', 0.70740219669265)

('Estoy en la iteracion', 4)
('El error total es:', 0.7011707542672008)

('Estoy en la iteracion', 5)
('El error total es:', 0.7036782855604653)

('Termine todo y la mejor iteracion fue con best_max_depth:', 3)


In [51]:
#Optimizo los 
best_score = 0
best_colsample_bylevel = 0
current_score=[]
for i in np.arange(0,6,1):
    print('Estoy en la iteracion',i)
    current_score = []
    for train_index,test_index in kf.split(train_sin_texto):
        x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
        y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
        xgc = xgb.XGBClassifier(n_estimators=170,learning_rate=0.6, reg_alpha=i)
        xgc.fit(x_train, y_train)
       
        #error de test:
        pred = xgc.predict(x_test)
        score = f1_score(y_test.values, pred, average='macro')
        current_score.append(score)
        
    print('El error total es:', sum(current_score)/len(current_score))
    if(sum(current_score)/len(current_score)>best_score):
        best_score = sum(current_score)/len(current_score)
        best_colsample_bylevel = i
    print('')
    

print('Termine todo y la mejor iteracion fue con best_colsample_bylevel:', best_colsample_bylevel) 

('Estoy en la iteracion', 0)
('El error total es:', 0.70740219669265)

('Estoy en la iteracion', 1)
('El error total es:', 0.705265705675848)

('Estoy en la iteracion', 2)
('El error total es:', 0.7050784069736327)

('Estoy en la iteracion', 3)
('El error total es:', 0.7005349270841756)

('Estoy en la iteracion', 4)
('El error total es:', 0.6959465720730288)

('Estoy en la iteracion', 5)
('El error total es:', 0.69093256246764)

('Termine todo y la mejor iteracion fue con best_colsample_bylevel:', 0)


In [42]:
for train_index,test_index in kf.split(train_sin_texto):
    x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
    y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
    xgc = xgb.XGBClassifier(n_estimators=170,learning_rate=0.6)
    xgc.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = xgc.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = xgc.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))
    
    
    
    
    #x_train = train.drop['target']
    #x_test = test.drop['target']
    #y_train = train.target
    #y_test = test.target
    #print(train,test)

('Error de entrenamiento: ', 0.8359642415113908)
('Error de test: ', 0.7336317707285449)
('Error de entrenamiento: ', 0.8441013684893299)
('Error de test: ', 0.6761443436445995)
('Error de entrenamiento: ', 0.8376785835896052)
('Error de test: ', 0.7059742429111355)
('Error de entrenamiento: ', 0.8404794289942442)
('Error de test: ', 0.69901834030762)
('Error de entrenamiento: ', 0.8337612462114494)
('Error de test: ', 0.7222422858713504)


In [54]:
# Entonces, mi modelo optimizado es:
train_x=train_limpio.drop(['text', 'target'], axis=1)

train_y=train_limpio.target



xgc = xgb.XGBClassifier(n_estimators=170,learning_rate=0.6)
xgc.fit(train_x, train_y)
    
#error de entrenamiento, es el unico que puedo calcular:
pred = xgc.predict(train_x)
print('Error de entrenamiento: ',f1_score(train_y, pred, average='macro'))

('Error de entrenamiento: ', 0.8218694822450909)


In [69]:
pred = xgc.predict(test_limpio_sin_texto.drop(['id'],axis=1))

In [70]:
sample.target=pred

In [71]:
sample.to_csv('prediccion_xgboost.csv', index=False)